In [1]:
import pandas as pd

review = pd.read_csv('fitbit_shap.csv', sep='\t', encoding = 'utf-8')
review.head()

,Iterator,Name,Title,Review,Rating,Prediction,Shap_json
0,0,Amazon Customer,Not worth the hassle,The strap is really hard to change. I managed ...,1.0,0.526897,"[{""shap"":-1.2216773556,""term"":""attach"",""occurr..."
1,1,Aubrey,Listen to the poor reviews,I've used fitbit products for years. My well l...,1.0,0.687864,"[{""shap"":-1.0183404569,""term"":""poor"",""occurren..."
2,2,Amazon Customer,It stopped working after 3 months,My fitbit stopped working after 3 months. The ...,2.0,2.005359,"[{""shap"":1.2231704767,""term"":""prompted"",""occur..."
3,3,Sophia,scratched and not that great,I have had this watch for less than a month an...,3.0,3.050132,"[{""shap"":1.3777308968,""term"":""alta"",""occurrenc..."
4,4,Cassandra Reising,All in all a good watch,I have had this for about a month and am very ...,4.0,3.999938,"[{""shap"":0.4017336522,""term"":""amazing"",""occurr..."


In [2]:
def getRating(input):
    if (input == 1):
        return 'one'
    elif (input == 2):
        return 'two'
    elif (input == 3):
        return 'three'
    elif (input == 4):
        return 'four'
    elif (input == 5):
        return 'five'
    
    return 'zero'

In [3]:
def getHeader(name, rating, title):
    r = getRating(rating)
    header = '<div class="single"><p><img src="avatar.jpg" /> <span class="reviewer_name">' + name + '</span></p><p><img src="trans.png" '
    next_line = 'class="star_rating ' + r + '" /><span class="review_title">' + title + '</span></p><p class="review_result">'
    
    return header + next_line

In [4]:
def getEndLine():
    return '</p></div>'

In [5]:
def getMiddle(text_review, shap_string):
    result = evaluate(text_review, shap_string)
    return result

In [6]:
import json

def parseShap(shap_string):
    return json.loads(shap_string)

In [7]:
def find_dictionary(list, term):    
    for x in range(0, len(list)):
        if (list[x]['term'] == term.lower()) or (term[0:-1].lower() == list[x]['term'] and term[-1] == 's'):
            return list[x]
        elif (term.lower() == "watches") and (list[x]['term'] == 'watch'):
            return list[x]
    
    return None

In [8]:
def evaluate(input, shap):
    parts = custom_split(input)
    output = ""
    for part in parts:
        parse = parseShap(shap)
        s = find_dictionary(parse, part)
        if s is not None:
            element = invoke_paragraph(s, part)
            output = output + element
        else: output = output + part
    return output

In [9]:
def custom_split(input):
    array = ['.', '?', '!', ',', ' ', '-', '"']
    result = []
    length = len(input)

    word = ''

    for i in range(len(input)):
        char = input[i]
        if char in array:
            if word != '' :
                result.append(word)
                word = ""
            result.append(char)
        else:
            word += char
            
    if word != ' ' : 
        result.append(word)

    return result

In [10]:
def invoke_paragraph(dictionary, part):
    factor = dictionary['factor']
    identifier = dictionary['id']
    orientation = dictionary['orientation']
        
    return highlight_part(factor, identifier, orientation, part)

In [11]:
def highlight_part(shapvalue, identifier, orientation, term) :
    factor = round(abs(shapvalue),2)
    orientation = lambda x : "positive" if (x > 0) else "negative"
    
    return "<span class=\"highlighter " + identifier + "\" data-shap=\"" + str(factor) + "\" data-orientation=\"" + orientation(shapvalue) + "\">" + term + "</span>";

In [12]:
name = ""
rating = ""
title = ""

html_col = []

for i in review.index:
    row = review.loc[i]
    
    shap = row.Shap_json
    name = row.Name
    rating = row.Rating #getRating(row.score)
    title = row.Title
    review_text = row.Review
    
    h = getHeader(name, rating, title)
    middle = getMiddle(review_text, shap)
    e = getEndLine()
    html = h + middle + e
    html_col.append(html)
    
    shap = ""
    name = ""
    rating = ""
    title = ""
    review_text = ""

In [13]:
review["Highlights"] = html_col
review.to_csv('fitbit_highlights.csv', sep='\t', index=False)
review

,Iterator,Name,Title,Review,Rating,Prediction,Shap_json,Highlights
0,0,Amazon Customer,Not worth the hassle,The strap is really hard to change. I managed ...,1.0,0.526897,"[{""shap"":-1.2216773556,""term"":""attach"",""occurr...","<div class=""single""><p><img src=""avatar.jpg"" /..."
1,1,Aubrey,Listen to the poor reviews,I've used fitbit products for years. My well l...,1.0,0.687864,"[{""shap"":-1.0183404569,""term"":""poor"",""occurren...","<div class=""single""><p><img src=""avatar.jpg"" /..."
2,2,Amazon Customer,It stopped working after 3 months,My fitbit stopped working after 3 months. The ...,2.0,2.005359,"[{""shap"":1.2231704767,""term"":""prompted"",""occur...","<div class=""single""><p><img src=""avatar.jpg"" /..."
3,3,Sophia,scratched and not that great,I have had this watch for less than a month an...,3.0,3.050132,"[{""shap"":1.3777308968,""term"":""alta"",""occurrenc...","<div class=""single""><p><img src=""avatar.jpg"" /..."
4,4,Cassandra Reising,All in all a good watch,I have had this for about a month and am very ...,4.0,3.999938,"[{""shap"":0.4017336522,""term"":""amazing"",""occurr...","<div class=""single""><p><img src=""avatar.jpg"" /..."
5,5,Gearhead,Works as designed.,Bought this for my girlfriend. Thought I was t...,4.0,4.057881,"[{""shap"":-0.4577140271,""term"":""apps"",""occurren...","<div class=""single""><p><img src=""avatar.jpg"" /..."
6,6,speedemonj,Great for fitness,"Great for fitness, sleep, etc but you have to ...",4.0,4.112775,"[{""shap"":0.31252238,""term"":""great"",""occurrence...","<div class=""single""><p><img src=""avatar.jpg"" /..."
7,7,Tyler,Good bang for your buck,I had a Fitbit blaze before I got the versa 2 ...,4.0,4.344578,"[{""shap"":-1.0821097325,""term"":""horrible"",""occu...","<div class=""single""><p><img src=""avatar.jpg"" /..."
8,8,Kindle Customer,HUGE improvement over the first Fitbit Versa,My first Fitbit Versa was great and died 4 mon...,5.0,4.502670,"[{""shap"":0.7109636365,""term"":""great"",""occurren...","<div class=""single""><p><img src=""avatar.jpg"" /..."
9,9,Nicole Quigley,Love It!!,After buying a cheaper type of fitness tracker...,5.0,4.955996,"[{""shap"":0.6609941128,""term"":""motivates"",""occu...","<div class=""single""><p><img src=""avatar.jpg"" /..."
